In [1]:
#importing necessary libraries
from tqdm import tqdm
import h5py
import numpy as np
import math

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

# Read the spotlists from a file

In [3]:
# Decomment to install crystfelparser

# ! git clone https://github.com/pgasparo/crystfelparser.git
# ! cd crystfelparser && pip install .
# !rm -rf crystfelparser

from crystfelparser.crystfelparser import stream_to_dictionary
from crystfelparser.utils import load_dict_from_hdf5

In [4]:
parsed=stream_to_dictionary("lyso_5_1000Hz_dtz170_data_000049.th5.snr5.0.mpixco1.mpeaks8.stream")

indexableframes=np.array(sorted([FR for FR in parsed.keys() if len(parsed[FR].keys())>7]))
indexableframes

array([ 118,  119,  120, ..., 9997, 9998, 9999])

In [6]:
# ratio of indexable frames
len(indexableframes)/len(parsed)

0.3938

# Some functions

In [31]:
def rodrigues(h,phi,rot_ax):
    import numpy as np
    
    cp=np.cos(phi)
    sp=np.sin(phi)
    omcp=1.-cp
    
    rot_h=np.zeros(3)
    
    rot_h[0] = (cp+rot_ax[0]**2*omcp)*h[0] + \
               (-rot_ax[2]*sp+rot_ax[0]*rot_ax[1]*omcp)*h[1] + \
               ( rot_ax[1]*sp+rot_ax[0]*rot_ax[2]*omcp)*h[2]
    rot_h[1] = ( rot_ax[2]*sp+rot_ax[0]*rot_ax[1]*omcp)*h[0] + \
               ( cp+rot_ax[1]**2*omcp)*h[1] + \
               (-rot_ax[0]*sp + rot_ax[1]*rot_ax[2]*omcp)*h[2]
    rot_h[2] = (-rot_ax[1]*sp+rot_ax[0]*rot_ax[2]*omcp)*h[0] + \
               ( rot_ax[1]*sp+rot_ax[1]*rot_ax[2]*omcp)*h[1] + \
               ( cp+rot_ax[2]**2*omcp)*h[2]
    
    return rot_h


def map_3D(spotslist,
           wavelength=0.999857,
           incident_beam=np.asarray([0.,0.,1.]),
           # size of the panel
           nx=4148,      
           ny=4362,
           # size of the pixels
           qx=0.075000,    
           qy=0.075000,
           orgx=2120.750488,
           orgy=2146.885498,
           det_dist=300.0,
           det_x=np.asarray([1.0,0.0,0.0]),
           det_y=np.asarray([0.0,1.0,0.0]),
           resolmax=0.1,
           resolmin=999,
           starting_angle=0,
           oscillation_range=0.0,
           rot_ax=np.asarray([1.000000,0.,0.])
          ):
    
    import numpy as np
    
    det_z=np.zeros(3)
    # comput z in case x,y are not perpendicular to the beam
    det_z[0]=det_x[1]*det_y[2]-det_x[2]*det_y[1] # calculate detector normal -
    det_z[1]=det_x[2]*det_y[0]-det_x[0]*det_y[2] # XDS.INP does not have
    det_z[2]=det_x[0]*det_y[1]-det_x[1]*det_y[0] # this item.
    det_z = det_z/np.sqrt(np.dot(det_z,det_z))   # normalize (usually not req'd)
    
    spots=[]

    for line in spotslist:
        (ih,ik,il)=(0.,0.,0.)
        if len(line)==4:
            (x,y,phi,intensity)=line
        elif len(line)==7:
            (x,y,phi,intensity,ih,ik,il)=line
        elif len(line)==3:
            (x,y,intensity)=line
            phi=0.0
        else:
            (x,y)=line
            phi=0.0
            intensity=0.0
    
        # convert detector coordinates to local coordinate system
        r= np.asarray([
            (x-orgx)*qx*det_x[0] + (y-orgy)*qy*det_y[0] +det_dist*det_z[0],
            (x-orgx)*qx*det_x[1] + (y-orgy)*qy*det_y[1] +det_dist*det_z[1],
            (x-orgx)*qx*det_x[2] + (y-orgy)*qy*det_y[2] +det_dist*det_z[2],
        ])
    
    
        # normalize scattered vector to obtain S1
        r=r/(wavelength*np.sqrt(np.dot(r,r)))
        # obtain reciprocal space vector S = S1-S0
        r=r-incident_beam
    
    
        if (np.sqrt(np.dot(r,r))>1./resolmax):
            continue # outer resolution limit
        if (np.sqrt(np.dot(r,r))<1./resolmin):
            continue # inner resolution limit
    
        # rotate  
        # NB: the term "-180." (found by trial&error) seems to make it match dials.rs_mapper
        phi=(starting_angle+oscillation_range*phi -180.)/180.*np.pi
    
        rot_r=rodrigues(r,phi,rot_ax)
    
        #rot_r=100.*rot_r + 100./resolmax  # ! transform to match dials.rs_mapper
    
        spots.append(np.hstack([rot_r,[intensity],[ih,ik,il]]))
        
    return np.asarray(spots)


def checkcell(outtext):
    from ase.cell import Cell
    if int(outtext[2].split()[1])>0:
        fullcell=np.asarray(list(map(np.float,(outtext[4]+" "+outtext[5]+" "+outtext[6]).split()))).reshape(3,3).T
        pdbcell=np.hstack([Cell(fullcell).lengths(), Cell(fullcell).angles()])
        
        return (fullcell,pdbcell)
    else:
        return [] 

# RUN ON SOME TEST FRAMES

In [37]:
cells=[]
pdbcells=[]
for FR in tqdm(indexableframes[:10]):
    STROUT=""
    STROUT+="{} 0 0 0 {} 0 0 0 {}\n".format(77.9,77.9,37.15)

    # GET THE PEAKS --- SUBSTITUTE THIS WITH YOUR SPOTFINDER!
    peaks=np.asarray([ll for ll in parsed[FR]['predicted_reflections'][:,[0,1,2]] if ll[-1]>0.5])
    #peaks=parsed[FR]['peaks']
    
    sp3d =  map_3D(   
          peaks,
          oscillation_range=0.0, # this is to not rotate the spots
          wavelength=0.999872983,
          nx=2068,      
          ny=2164,
          qx=0.075,
          qy=0.075,
          orgx=1133.66,
          orgy=1171.89,
          det_dist=170,
    )[:,:3]
    
    for pk in sp3d:
        STROUT+="{:.8f} {:.8f} {:.8f}\n".format(*pk)
        
    # open a tmp text file
    filename="{}".format(FR)+".3D"
    text_file = open(filename, "w")
    # write string to file
    _ = text_file.write(STROUT)
    # close file
    text_file.close()
    
    # try to index the points
    import subprocess
    
    # call xgandalf
    proc = subprocess.Popen("/das/home/gasparotto_p/bin/xgandalfHCS {}".format(filename), stdout=subprocess.PIPE, shell=True)
    (out, err) = proc.communicate()
    out=out.decode('UTF-8')
    
    # interpret the output
    tmpcell=checkcell(out.split("\n"))
    
    if len(tmpcell)==2:
        cells.append(tmpcell[0])
        pdbcells.append(tmpcell[1])
    else:
        cells.append([])
        pdbcells.append([])
    
    # remove the tmp file
    subprocess.call(['rm','-r',filename])

100%|██████████| 10/10 [00:02<00:00,  3.87it/s]


# CHECK THE INDEXED CELLS

In [38]:
pdbcells

[array([37.21755345, 77.83706629, 77.875633  , 89.65717547, 89.98263167,
        89.01073151]),
 array([37.1697798 , 77.83006097, 77.86804838, 89.63700287, 91.24650377,
        90.10288093]),
 array([37.19044117, 77.89307643, 77.90931532, 90.3405341 , 89.72301129,
        91.17176594]),
 array([37.15305227, 77.69003702, 77.73151602, 90.10653313, 89.89488337,
        89.267012  ]),
 array([37.25311685, 77.78828713, 77.87348933, 89.67781444, 89.25246641,
        90.16363532]),
 array([37.32084   , 77.84891016, 77.96947911, 90.30622277, 90.98858868,
        90.10492903]),
 array([37.18456255, 77.90515696, 78.06325324, 90.29742224, 91.11773223,
        89.99018017]),
 array([37.1680865 , 77.85660676, 77.92937617, 90.46016602, 89.84221936,
        91.17919951]),
 array([37.1859587 , 77.82880548, 77.96150981, 89.62188156, 89.75434944,
        88.8392701 ]),
 array([37.21086196, 77.78006118, 77.93135961, 90.27155847, 89.82472278,
        91.18278641])]